## Вбудовування

У нашому попередньому прикладі ми працювали з високовимірними векторами "мішок слів" довжиною `vocab_size`, і ми явно перетворювали низьковимірні вектори позиційного представлення у розріджене одноелементне представлення. Таке одноелементне представлення не є ефективним з точки зору пам'яті, до того ж кожне слово розглядається незалежно одне від одного, тобто одноелементні закодовані вектори не виражають жодної семантичної схожості між словами.

У цьому розділі ми продовжимо досліджувати набір даних **News AG**. Для початку завантажимо дані та отримаємо деякі визначення з попереднього блокнота.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Що таке вбудовування?

Ідея **вбудовування** полягає в тому, щоб представляти слова за допомогою низьковимірних щільних векторів, які певним чином відображають семантичне значення слова. Пізніше ми обговоримо, як створювати змістовні вбудовування слів, але наразі давайте просто розглядати вбудовування як спосіб зменшення розмірності векторів слів.

Отже, шар вбудовування приймає слово як вхід і створює вихідний вектор заданого розміру `embedding_size`. У певному сенсі це дуже схоже на шар `Linear`, але замість того, щоб приймати вектор з одним активним елементом (one-hot encoded vector), він може приймати номер слова як вхід.

Використовуючи шар вбудовування як перший шар у нашій мережі, ми можемо перейти від моделі "мішок слів" до моделі **мішок вбудовувань**, де спочатку кожне слово в тексті перетворюється на відповідне вбудовування, а потім обчислюється певна агрегатна функція для всіх цих вбудовувань, наприклад `sum`, `average` або `max`.

![Зображення, що показує класифікатор вбудовувань для п’яти слів у послідовності.](../../../../../translated_images/uk/embedding-classifier-example.b77f021a7ee67eee.webp)

Наша нейронна мережа класифікатора починатиметься з шару вбудовування, потім шару агрегування, а зверху — лінійного класифікатора:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Робота зі змінним розміром послідовності змінних

Через особливості цієї архітектури, мініпакети для нашої мережі потрібно створювати певним чином. У попередньому розділі, коли ми використовували метод "мішок слів" (bag-of-words), усі тензори BoW у мініпакеті мали однаковий розмір `vocab_size`, незалежно від фактичної довжини текстової послідовності. Коли ми переходимо до використання словникових векторів (word embeddings), кількість слів у кожному текстовому зразку стає змінною, і при об'єднанні цих зразків у мініпакети нам доведеться застосовувати заповнення (padding).

Це можна зробити за допомогою тієї ж техніки, що й надання функції `collate_fn` джерелу даних:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Навчання класифікатора на основі вбудовування

Тепер, коли ми визначили відповідний завантажувач даних, ми можемо навчити модель, використовуючи функцію навчання, яку ми визначили в попередньому розділі:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Примітка**: Ми тренуємося лише на 25 тисячах записів (менше одного повного епоху) заради економії часу, але ви можете продовжити тренування, написати функцію для тренування протягом кількох епох і експериментувати з параметром швидкості навчання, щоб досягти вищої точності. Ви повинні бути здатні досягти точності близько 90%.


### Шар EmbeddingBag та представлення послідовностей змінної довжини

У попередній архітектурі нам потрібно було доповнювати всі послідовності до однакової довжини, щоб вони відповідали розміру мініпакету. Це не найефективніший спосіб представлення послідовностей змінної довжини — інший підхід полягає у використанні **вектора зсувів**, який містить зсуви всіх послідовностей, збережених в одному великому векторі.

![Зображення, що показує представлення послідовності за допомогою зсувів](../../../../../translated_images/uk/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Примітка**: На зображенні вище показано послідовність символів, але в нашому прикладі ми працюємо з послідовностями слів. Однак загальний принцип представлення послідовностей за допомогою вектора зсувів залишається тим самим.

Для роботи з представленням за допомогою зсувів ми використовуємо шар [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Він схожий на `Embedding`, але приймає вектор вмісту та вектор зсувів як вхідні дані, а також включає шар усереднення, який може бути `mean`, `sum` або `max`.

Ось модифікована мережа, яка використовує `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Щоб підготувати набір даних для навчання, нам потрібно надати функцію конвертації, яка підготує вектор зміщення:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Зверніть увагу, що, на відміну від усіх попередніх прикладів, наша мережа тепер приймає два параметри: вектор даних і вектор зміщення, які мають різні розміри. Аналогічно, наш завантажувач даних також надає нам 3 значення замість 2: як текстові, так і вектори зміщення надаються як ознаки. Тому нам потрібно трохи змінити нашу функцію навчання, щоб врахувати це:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Семантичні Вбудовування: Word2Vec

У нашому попередньому прикладі шар вбудовування моделі навчався відображати слова у векторне представлення, однак це представлення не мало значного семантичного значення. Було б добре навчитися створювати таке векторне представлення, де схожі слова або синоніми відповідали б векторами, які близькі один до одного за певною векторною відстанню (наприклад, евклідовою відстанню).

Для цього нам потрібно попередньо навчити нашу модель вбудовування на великій колекції текстів у специфічний спосіб. Один із перших методів навчання семантичних вбудовувань називається [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Він базується на двох основних архітектурах, які використовуються для створення розподіленого представлення слів:

 - **Неперервний мішок слів** (CBoW) — у цій архітектурі ми навчаємо модель передбачати слово за навколишнім контекстом. Для даного n-граму $(W_{-2},W_{-1},W_0,W_1,W_2)$ мета моделі — передбачити $W_0$ за $(W_{-2},W_{-1},W_1,W_2)$.
 - **Неперервний skip-gram** є протилежністю до CBoW. Модель використовує навколишнє вікно контекстних слів, щоб передбачити поточне слово.

CBoW працює швидше, тоді як skip-gram повільніший, але краще справляється з представленням рідковживаних слів.

![Зображення, що демонструє алгоритми CBoW та Skip-Gram для перетворення слів у вектори.](../../../../../translated_images/uk/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Щоб експериментувати з вбудовуванням word2vec, попередньо навченим на наборі даних Google News, ми можемо використовувати бібліотеку **gensim**. Нижче наведено приклад пошуку слів, найбільш схожих на 'neural'.

> **Примітка:** Коли ви вперше створюєте векторні представлення слів, їх завантаження може зайняти деякий час!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Ми також можемо обчислити векторні вбудовування зі слова, щоб використовувати їх у навчанні моделі класифікації (ми показуємо лише перші 20 компонентів вектора для ясності):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Чудова річ у семантичних вбудовуваннях полягає в тому, що ви можете маніпулювати векторним кодуванням, щоб змінити семантику. Наприклад, ми можемо попросити знайти слово, чия векторна репрезентація буде якомога ближчою до слів *король* і *жінка*, і якомога далі від слова *чоловік*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Обидва методи, CBoW і Skip-Grams, є "прогнозуючими" векторними уявленнями, оскільки вони враховують лише локальні контексти. Word2Vec не використовує глобальний контекст.

**FastText** розширює Word2Vec, навчаючи векторні уявлення для кожного слова та n-грам символів, які зустрічаються в межах кожного слова. Значення цих уявлень потім усереднюються в один вектор на кожному кроці навчання. Хоча це додає значну кількість додаткових обчислень під час попереднього навчання, це дозволяє векторним уявленням слів кодувати інформацію про частини слова.

Інший метод, **GloVe**, використовує ідею матриці спільної зустрічальності, застосовуючи нейронні методи для розкладання цієї матриці на більш виразні та нелінійні векторні уявлення слів.

Ви можете експериментувати з прикладом, змінюючи векторні уявлення на FastText і GloVe, оскільки gensim підтримує кілька різних моделей векторних уявлень слів.


## Використання попередньо навчених векторів у PyTorch

Ми можемо змінити наведений вище приклад, щоб попередньо заповнити матрицю у нашому шарі векторизації семантичними векторами, такими як Word2Vec. Потрібно врахувати, що словники попередньо навчених векторів і нашого текстового корпусу, ймовірно, не співпадатимуть, тому ми ініціалізуємо ваги для відсутніх слів випадковими значеннями:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Тепер давайте навчимо нашу модель. Зверніть увагу, що час, необхідний для навчання моделі, значно більший, ніж у попередньому прикладі, через більший розмір шару вбудовування, а отже, значно більшу кількість параметрів. Також через це нам може знадобитися навчати нашу модель на більшій кількості прикладів, якщо ми хочемо уникнути перенавчання.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

У нашому випадку ми не бачимо значного підвищення точності, що, ймовірно, пов’язано з досить різними словниками.  
Щоб вирішити проблему різних словників, ми можемо скористатися одним із наступних рішень:  
* Перенавчити модель word2vec на нашому словнику  
* Завантажити наш набір даних із використанням словника з попередньо навченої моделі word2vec. Словник, який використовується для завантаження набору даних, можна вказати під час завантаження.  

Останній підхід здається простішим, особливо тому, що фреймворк PyTorch `torchtext` містить вбудовану підтримку для векторів слів. Наприклад, ми можемо створити словник на основі GloVe наступним чином:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Завантажений словник має наступні базові операції:
* Словник `vocab.stoi` дозволяє перетворити слово на його індекс у словнику
* `vocab.itos` робить протилежне - перетворює число на слово
* `vocab.vectors` є масивом векторів вбудовування, тому, щоб отримати вбудовування слова `s`, нам потрібно використати `vocab.vectors[vocab.stoi[s]]`

Ось приклад маніпуляції з вбудовуваннями, щоб продемонструвати рівняння **kind-man+woman = queen** (я трохи підкоригував коефіцієнт, щоб це спрацювало):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Щоб навчити класифікатор за допомогою цих векторів, спочатку потрібно закодувати наш набір даних за допомогою словника GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Як ми бачили вище, всі векторні вбудовування зберігаються в матриці `vocab.vectors`. Це робить надзвичайно простим завантаження цих ваг у ваги шару вбудовування за допомогою простого копіювання:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Однією з причин, чому ми не спостерігаємо значного підвищення точності, є те, що деякі слова з нашого набору даних відсутні у попередньо натренованому словнику GloVe, і тому вони фактично ігноруються. Щоб подолати це, ми можемо натренувати власні векторні представлення на нашому наборі даних.


## Контекстуальні векторні представлення

Одним із ключових обмежень традиційних попередньо навчених векторних представлень, таких як Word2Vec, є проблема розрізнення значень слів. Хоча попередньо навчені вектори можуть захоплювати певний зміст слів у контексті, усі можливі значення слова кодуються в одному й тому ж векторі. Це може спричиняти проблеми в подальших моделях, оскільки багато слів, наприклад слово "play", мають різні значення залежно від контексту, у якому вони використовуються.

Наприклад, слово "play" у цих двох реченнях має зовсім різні значення:
- Я пішов на **п'єсу** в театрі.
- Джон хоче **грати** зі своїми друзями.

Попередньо навчені вектори, згадані вище, представляють обидва ці значення слова "play" в одному й тому ж векторі. Щоб подолати це обмеження, потрібно створювати векторні представлення на основі **мовної моделі**, яка тренується на великому корпусі тексту і *знає*, як слова можуть поєднуватися в різних контекстах. Обговорення контекстуальних векторних представлень виходить за рамки цього уроку, але ми повернемося до них, коли будемо говорити про мовні моделі в наступному розділі.



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
